In [ ]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 51 kB 185 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=b7949770e984c3cd391f571f75e28ede5334b190528affca50152c7ce686b73b
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("4JIZFHOTUEMJYMAJ2CVRZY4U-7M5TIFUVU3WS3IDM")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# used a dictionary to represent an intents JSON file
data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi", "Whats up"],
              "responses": ["Hello"]
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when was you born?"],
              "responses": ["I am 24 years old", "I was born in 1996", "My birthday is July 3rd 2022"]
             },
             {"tag": "date",
              "patterns": ["what are you doing this weekend?",
"do you want to hang out some time?", "what are your plans for this week"],
              "responses": ["I am available all week", "I don't have any plans"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?","name?"],
              "responses": ["My name is Nanbi"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "stop", "see ya", "adios", "Thx"],
              "responses": ["It was nice speaking to you. See you later."]
             },
             {"tag": "loan",
              "patterns": [ "loan?", "loan available?", "What am i eligible for?", "loans", "farmer loan"],
              "responses": ["I am searching for top loan plans suited for you."]
             },
             {"tag": "loan",
              "patterns": [ "loan?", "loan available?", "What am i eligible for?", "loans", "farmer loan"],
              "responses": ["I am searching for top loan plans suited for you."]
             }
]}

In [ ]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5888      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 14,534
Trainable params: 14,534
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [==============================

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 1.3 MB 10.5 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=238f4cb27ed200914477cf7c22aaaf5a8fc5237c589358018ff5bc468718e3ca
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
import googletrans
translator = googletrans.Translator()
a="நீங்கள் எப்படி இருக்கிறீர்கள்"
translated = translator.translate(a,src='ta',dest='en')
print(translated.text)

How are you


In [ ]:
pip install pyPDF4

     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61248 sha256=1e71ba195d80a09e7965afe660683c110135462c56ad5cf02914435ac24c5b7f
  Stored in directory: /root/.cache/pip/wheels/82/db/a8/ab40e2699944d458726831fed55791a14d800e0dc5a4889869
Successfully built pyPDF4


In [ ]:
pip install tabula-py

     |████████████████████████████████| 12.0 MB 5.3 MB/s 


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import re
import tabula
import pandas as pd
from PyPDF4 import PdfFileReader
def docsearch(*loan):
  file1 = "/content/gdrive/MyDrive/loan doc.pdf"
  index = tabula.read_pdf(file1,pages=1)
  table=list(index[0]["Loan"])
  table = [i.lower() for i in table]
  print(table)
  if(len(loan)!=0):
    a=table.index(loan[0]+" loan")
    page=index[0]["Pg.no"][a]
    temp = open(file1, 'rb')
    PDF_read = PdfFileReader(temp)
    req_page = PDF_read.getPage(page-1)
    req=(req_page.extractText())
    #for line in req.split('/n'):
    return req
  else:
    return index

In [ ]:
# running the chatbot
#- *- coding: utf- 8 - *-
import re
@anvil.server.callable
def sendmessage(message):
  list1 = ["farmer","personal","student","home", "business"]
  #while message != "stop":
      #message = input().lower()# Create translation model
  translated = translator.translate(message,dest='en')
  #print(translated.text)
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  translated = translator.translate(result,src='en',dest='ta')
  ret = translated.text
  if(intents[0]=="loan"):
        #rint("entered")
      for i in list1:
          x = re.search(i,message)
          #print(x,x!=None,sep="-")
          if(x!=None):
            print(x.span())
            ret =  translated.text+docsearch(i)
            break
          else:
            ret = "No loans schemes available similar to your request"
  return ret
anvil.server.wait_forever()

KeyboardInterrupt: ignored

In [ ]:
sendmessage("can i get farmers loan")